# Main Aggregation Script

In [1]:
#Reloads the lab.py and crystals.py modules to update any changes (after saving)
#If a new method or object is created, autoreload doesn't work and the 
#kernel needs to be closed and halted after saving and making a 'checkpoint'
#in this notebook


%load_ext autoreload
%autoreload 2

In [2]:
import ipas
import matplotlib.pyplot as plt
import numpy as np
from sqlite3 import Connection as SQLite3Connection
from sqlalchemy import create_engine, event, select
from sqlalchemy.engine import Engine
from sqlalchemy.orm import sessionmaker
import base
from ipas import lab_ice_agg_SQL as lab
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
from dask import delayed
import cloudpickle as pickle
from multiprocessing import Pool
from functools import partial
import logging
import time

In [3]:
cluster = SLURMCluster(
    queue='kratos',
    walltime='04-23:00:00',
    cores=1,
    memory='7168MiB', #1 GiB = 1,024 MiB
    processes=1)


cluster.scale(20)

/network/rit/lab/sulialab/share/bin/miniconda3/envs/pangeo/lib/python3.6/site-packages/distributed/deploy/local.py:138: UserWarning: diagnostics_port has been deprecated. Please use `dashboard_address=` instead
  "diagnostics_port has been deprecated. "


In [4]:
client = Client(cluster)

In [8]:
client

Client Scheduler: tcp://169.226.65.47:42663 Dashboard: http://169.226.65.47:8787/status,Cluster Workers: 20 Cores: 20 Memory: 150.40 GB


In [3]:
phioarr=np.logspace(-2, 2, num=20, dtype=None)#just columns (0,2); plates (-2,0)
#phioarr = phioarr[1:]
lenphio = len(phioarr)

reqarr = [50,60,70,80]
numaspectratios=len(phioarr)
ch_dist='gamma'         #anything other than gamma uses the characteristic from the best distribution pdf (lowest SSE)
nclusters = 300        #changes how many aggregates per aspect ratio to consider
ncrystals = 50       #number of monomers per aggregate 1
minor = 'depth'        #'minorxy' from fit ellipse or 'depth' to mimic IPAS in IDL
rand_orient = True    #randomly orient the seed crystal and new crystal: uses first random orientation
save_plots = False     #saves all histograms to path with # of aggs and minor/depth folder
file_ext = 'eps'

In [4]:
def main():
 
    '''
    for phio in phioarr:
        for r in reqarr:
            b1 = lab.collect_clusters(phio, r, nclusters=nclusters,
                                ncrystals=ncrystals, rand_orient=rand_orient)     
   
    output = []
    for r in reqarr:
        for phio in phioarr:
    
            print('eq. vol rad', r, phio)
            output.append(delayed(lab.collect_clusters)(phio, r, nclusters=nclusters,
                                                        ncrystals=ncrystals, rand_orient=rand_orient))
    
    print(output)
    print('computing...')
    b1 = client.compute(output) 
    b1 = client.gather(b1)
    '''

    notebook=3
    print(notebook)
    for r in reqarr:
        #b3 = []
        print('r = ',r)
        count = 1

        pool = Pool(processes=20) #pools are reusable
        parallel_clus=partial(lab.collect_clusters, notebook=notebook, r=r, nclusters=nclusters,\
                                ncrystals=ncrystals, \
                                rand_orient=rand_orient)

        start = time.time()
        output = pool.imap_unordered(parallel_clus, phioarr)
        for done in output:
            print("after %3.1fsec: count:%s"%(time.time()-start, count))
            count +=1

#         for res in output:
#             print("(after %3.1fsec)  mono phi:%.3f  count:%s"%(time.time()-start, res[0].mono_phi, count))
#             count += 1
        print('closing')
        pool.close()
        print('joining')
        pool.join()

        
        print('new r')



In [ ]:
if __name__ == '__main__':

    %time main() 
    
#     filename = 'instance_3radii_iceagg_allrand_50xtalstot_20phi_rall_300agg_lastmono'
#     filehandler = open(filename, 'wb')
#     %time pickle.dump(b, filehandler)
#     filehandler.close()
#     print('finished, creating engine 1!')
    
#     engine = create_engine('sqlite:///IPAS_lastmono.sqlite')
#     event.listen(engine, 'connect', _set_sqlite_pragma)
#     base.Base.metadata.create_all(engine, checkfirst=True)
#     Session = sessionmaker(bind=engine)
#     session = Session()
    
#     try:
#         for r in b:
#             for obj in r:
#                 session.add_all(obj)  # crystal id has been appended into cluster relationship
#                 session.commit()

#     except:
#         print('in except')
#         raise
#     session.close() 
#     print('DONE!')
    
    #packed = msgpack.writeResult('msgpack_test', b1)
    #msgpack.unpackb(packed, encoding='utf-8')
    
    #filename = 'instance_3radii_iceagg_allrand_100xtalstot_20phi_r10_1000agg'
    #filehandler = open(filename, 'wb')
    #%time pickle.dump(b1, filehandler)
    #filehandler.close()
    #print('finished!')
    
    #CHANGE LOOKUP TABLE FILES TO HAVE RANGE OF PHI AND R
    #Kratos = 28*4nodes = 112 -> 28 req so 4 aspect ratios at a time
    #would take 5 runs

3
r =  50
nclus 50 100.0 0
nclus 50 0.026366508987303583 0
nclus 50 8.858667904100823 0
nclus 50 1.2742749857031335 0
nclus 50 2.06913808111479 0
nclus 50 61.584821106602604 0
nclus 50 37.92690190732246 0
nclus 50 0.016237767391887217 0
nclus 50 5.455594781168514 0
nclus 50 0.29763514416313175 0
nclus 50 0.11288378916846889 0
nclus 50 3.359818286283781 0
nclus 50 14.38449888287663 0
nclus 50 0.04281332398719394 0
nclus 50 0.4832930238571752 0
nclus 50 0.7847599703514611 0
nclus 50 23.357214690901213 0
nclus 50 0.06951927961775606 0
nclus 50 0.18329807108324356 0
nclus 50 0.18329807108324356 20
nclus 50 0.06951927961775606 20
nclus 50 0.016237767391887217 20
nclus 50 0.29763514416313175 20
nclus 50 0.04281332398719394 20
nclus 50 0.026366508987303583 20
nclus 50 0.11288378916846889 20
nclus 50 0.7847599703514611 20
nclus 50 3.359818286283781 20
nclus 50 2.06913808111479 20
nclus 50 0.4832930238571752 20
nclus 50 1.2742749857031335 20
nclus 50 5.455594781168514 20
nclus 50 8.858667904100

done committing phio = 0.043
after 30656.8sec: count:3
done committing phio = 0.070
after 30666.9sec: count:4
done committing phio = 0.026
after 30700.9sec: count:5
done committing phio = 0.183
after 30794.4sec: count:6
done committing phio = 0.483
after 30805.4sec: count:7
done committing phio = 0.785
after 30834.0sec: count:8
done committing phio = 0.113
after 30839.3sec: count:9
nclus 50 61.584821106602604 180
nclus 50 37.92690190732246 200
nclus 50 23.357214690901213 220
done committing phio = 1.274
after 31075.1sec: count:10
nclus 50 5.455594781168514 280
done committing phio = 2.069
after 31239.0sec: count:11
done committing phio = 3.360
after 31456.7sec: count:12
nclus 50 8.858667904100823 280
nclus 50 14.38449888287663 260
done committing phio = 5.456
after 32144.0sec: count:13
nclus 50 100.0 180
nclus 50 23.357214690901213 240
nclus 50 37.92690190732246 220
nclus 50 61.584821106602604 200
done committing phio = 8.859
after 32720.5sec: count:14
nclus 50 14.38449888287663 280
nc

In [ ]:
def create_engines(eng_name):
    engine=create_engine(eng_name)
    db = 'SELECT * FROM aggregates'
    df = pd.read_sql(db, con=engine)
    return df

# databases = ['sqlite:///IPAS.sqlite', 'sqlite:///IPAS_1.sqlite', 'sqlite:///IPAS_2.sqlite',\
#             'sqlite:///IPAS_3.sqlite', 'sqlite:///IPAS_4.sqlite', 'sqlite:///IPAS_5.sqlite']

databases = ['sqlite:///IPAS.sqlite', 'sqlite:///IPAS_1.sqlite']

pool = Pool(processes=2)
df_all = pool.map(create_engines, databases)
pool.close()

# engine = create_engine('sqlite:///IPAS.sqlite')
# engine1 = create_engine('sqlite:///IPAS_1.sqlite')
# engine2 = create_engine('sqlite:///IPAS_2.sqlite')
# engine3 = create_engine('sqlite:///IPAS_3.sqlite')
# engine4 = create_engine('sqlite:///IPAS_4.sqlite')
# engine5 = create_engine('sqlite:///IPAS_5.sqlite')

# db = 'SELECT * FROM aggregates'
# df = pd.read_sql(db, con=engine)
# df1 = pd.read_sql(db, con=engine1)
# df2 = pd.read_sql(db, con=engine2)
# df3 = pd.read_sql(db, con=engine3)
# df4 = pd.read_sql(db, con=engine4)
# df5 = pd.read_sql(db, con=engine5)


In [92]:
%time df = dd.read_sql_table('aggregates', 'sqlite:///IPAS.sqlite',\
                       npartitions=10, index_col='id').persist()  

CPU times: user 4.53 s, sys: 4.09 s, total: 8.62 s
Wall time: 1min


In [93]:
%time df_1 = dd.read_sql_table('aggregates', 'sqlite:///IPAS_1.sqlite',\
                       npartitions=28, index_col='id').persist()
progress(df_1)

CPU times: user 6.4 s, sys: 5.23 s, total: 11.6 s
Wall time: 1min 25s


In [94]:
%time df_2 = dd.read_sql_table('aggregates', 'sqlite:///IPAS_2.sqlite',\
                       npartitions=28, index_col='id')
df_2.persist() 
progress(df_2)

CPU times: user 11 s, sys: 9.58 s, total: 20.5 s
Wall time: 2min 20s


VBox()

In [98]:
%time df_3 = dd.read_sql_table('aggregates', 'sqlite:///IPAS_3.sqlite',\
                       npartitions=28, index_col='id')
df_3.persist() 
progress(df_3)

CPU times: user 11.7 s, sys: 9.45 s, total: 21.2 s
Wall time: 2min 32s


NameError: name 'df3' is not defined

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x7fe80eb3de18>, <Future finished exception=OSError("Timed out trying to connect to 'tcp://10.77.9.102:42577' after 10 s: connect() didn't finish in time",)>)
Traceback (most recent call last):
  File "/network/rit/lab/sulialab/share/bin/miniconda3/envs/pangeo/lib/python3.6/site-packages/distributed/comm/core.py", line 210, in connect
    quiet_exceptions=EnvironmentError,
  File "/network/rit/lab/sulialab/share/bin/miniconda3/envs/pangeo/lib/python3.6/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
tornado.util.TimeoutError: Timeout

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/network/rit/lab/sulialab/share/bin/miniconda3/envs/pangeo/lib/python3.6/site-packages/tornado/ioloop.py", line 758, in _run_callback
    ret = callback()
  File "/network/rit/lab/sulialab/share/bin/miniconda

In [99]:
df_3.persist() 
progress(df_3)

VBox()

tornado.application - ERROR - Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x7fe7c8d7b048>, <Future finished exception=AttributeError("'MultiProgressWidget' object has no attribute 'elapsed_time'",)>)
Traceback (most recent call last):
  File "/network/rit/lab/sulialab/share/bin/miniconda3/envs/pangeo/lib/python3.6/site-packages/tornado/ioloop.py", line 758, in _run_callback
    ret = callback()
  File "/network/rit/lab/sulialab/share/bin/miniconda3/envs/pangeo/lib/python3.6/site-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/network/rit/lab/sulialab/share/bin/miniconda3/envs/pangeo/lib/python3.6/site-packages/tornado/ioloop.py", line 779, in _discard_future_result
    future.result()
  File "/network/rit/lab/sulialab/share/bin/miniconda3/envs/pangeo/lib/python3.6/site-packages/tornado/gen.py", line 1147, in run
    yielded = self.gen.send(value)
  File "/network/rit/lab/sulialab/share/bin/minicon

In [96]:
%time df_4 = dd.read_sql_table('aggregates', 'sqlite:///IPAS_4.sqlite',\
                       npartitions=28, index_col='id')
df_4.persist() 
progress(df_4)

CPU times: user 4.06 s, sys: 4.02 s, total: 8.09 s
Wall time: 55.6 s


VBox()

In [97]:
%time df_5 = dd.read_sql_table('aggregates', 'sqlite:///IPAS_5.sqlite',\
                       npartitions=28, index_col='id')
df_5.persist() 
progress(df_5)

CPU times: user 6.53 s, sys: 5.49 s, total: 12 s
Wall time: 1min 23s


VBox()

In [103]:
df_list = [df_1, df_2, df_3, df_4, df_5]
for c, db in enumerate(df_list):
    print(c)
    df.append(db).compute()

0


KeyboardInterrupt: 

In [66]:
df.tail()

,ncrystals,points,agg_phi,agg_r,a,b,c,cplx,phi2D
id,,,,,,,,,
2057996,46,b'\x80\x04\x95\x984\x00\x00\x00\x00\x00\x00\x8...,0.683566,3583.920129,2781.071423,3334.064825,4068.472759,0.996377,0.686009
2057997,47,b'\x80\x04\x95\xb85\x00\x00\x00\x00\x00\x00\x8...,0.683586,3584.111660,2781.272936,3333.655966,4068.651501,0.996496,0.664038
2057998,48,b'\x80\x04\x95\xd86\x00\x00\x00\x00\x00\x00\x8...,0.683570,3583.835018,2781.015942,3334.345052,4068.368413,0.996634,0.648678
2057999,49,b'\x80\x04\x95\xf87\x00\x00\x00\x00\x00\x00\x8...,0.683605,3551.110206,2755.714387,3390.472723,4031.151534,0.996740,0.633507
2058000,50,b'\x80\x04\x95\x189\x00\x00\x00\x00\x00\x00\x8...,0.683341,3553.748470,2757.051379,3377.954724,4034.666096,0.996842,0.625880


In [59]:
len(df),len(df_1),len(df_2),len(df_3)

(882000, 1176000, 2058000, 2058000)

In [53]:
agg_stats = client.compute(df['agg_phi'].describe().round(2))
progress(agg_stats)

VBox()

In [12]:
rand_orient = df.crystals.rand_orient
rand_orient_cat = rand_orient.astype('category')
print(rand_orient_cat.head())
print(mem_usage(rand_orient))
print(mem_usage(rand_orient_cat))

AttributeError: 'DataFrame' object has no attribute 'crystals'

In [5]:
df_all = pd.concat([df,df1,df2,df3,df4,df5])
print(len(result))

8234400


In [6]:
df_all.memory_usage()

Index        65875200
id           65875200
ncrystals    65875200
points       65875200
agg_phi      65875200
agg_r        65875200
a            65875200
b            65875200
c            65875200
cplx         65875200
phi2D        65875200
dtype: int64

In [11]:
df.info

<bound method DataFrame.info of Dask DataFrame Structure:
               ncrystals  points  agg_phi    agg_r        a        b        c     cplx    phi2D
npartitions=10                                                                                 
1.0                int64  object  float64  float64  float64  float64  float64  float64  float64
88200.9              ...     ...      ...      ...      ...      ...      ...      ...      ...
...                  ...     ...      ...      ...      ...      ...      ...      ...      ...
793800.1             ...     ...      ...      ...      ...      ...      ...      ...      ...
882000.0             ...     ...      ...      ...      ...      ...      ...      ...      ...
Dask Name: from-delayed, 20 tasks>

In [39]:
# engine = create_engine('sqlite:///IPAS_1.sqlite')
# base.Base.metadata.create_all(engine, checkfirst=True)
# Session = sessionmaker(bind=engine)
# session = Session()

engine = create_engine('sqlite:///IPAS_bkup.sqlite', echo=True)
engine.execute("attach database 'IPAS_1.sqlite' as d1;")
engine.execute("attach database 'IPAS_2.sqlite' as d2;")
engine.execute("attach database 'IPAS_3.sqlite' as d3;")
engine.execute("attach database 'IPAS_4.sqlite' as d4;")
engine.execute("attach database 'IPAS_5.sqlite' as d5;")
  
Session = sessionmaker(bind=engine)


session = Session()
inspector = Inspector.from_engine(engine)
print(inspector.get_table_names())


2019-12-02 15:28:45,390 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-12-02 15:28:45,392 INFO sqlalchemy.engine.base.Engine ()
2019-12-02 15:28:45,394 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-12-02 15:28:45,396 INFO sqlalchemy.engine.base.Engine ()
2019-12-02 15:28:45,398 INFO sqlalchemy.engine.base.Engine attach database 'IPAS_1.sqlite' as d1;
2019-12-02 15:28:45,400 INFO sqlalchemy.engine.base.Engine ()
2019-12-02 15:28:45,420 INFO sqlalchemy.engine.base.Engine attach database 'IPAS_2.sqlite' as d2;
2019-12-02 15:28:45,422 INFO sqlalchemy.engine.base.Engine ()
2019-12-02 15:28:45,428 INFO sqlalchemy.engine.base.Engine attach database 'IPAS_3.sqlite' as d3;
2019-12-02 15:28:45,430 INFO sqlalchemy.engine.base.Engine ()
2019-12-02 15:28:45,435 INFO sqlalchemy.engine.base.Engine attach database 'IPAS_4.sqlite' as d4;
2019-12-02 15:28:45,437 INFO sqlalchemy.engine.base.Engine (

In [ ]:
engines = []
sessions = []
databases=[IPAS.sqlite, IPAS_1.sqlite, IPAS_2.sqlite, IPAS_3.sqlite, IPAS_4.sqlite, IPAS_5.sqlite]
for dbconninfo in databases:
    engine = create_engine(dbconninfo)
    engines.append(engine)
    sessions.append(sessionmaker(bind=engine)())

In [35]:
print('AGGREGATE PROPERTIES ==============')

#print(session.query(ipas.IceCluster).count())#, crys.IceCrystal)#.filter(crys.IceCrystal.id == 10)

#print(session.query(ipas.IceCluster).filter(ipas.IceCrystal.r == 7).count())
#query = session.query(ipas.IceCluster).filter(ipas.IceCrystal.r == 10).limit(10)

for value in session.query(ipas.IceCrystal.r).distinct():
    print(value)

#for agg in query:
#    print(agg.id, agg.ncrystals, agg.agg_phi, agg.cplx, agg.crystal[0].id,  agg.crystal[0].phi, agg.crystal[0].r)

#print('MONOMER PROPERTIES ================')
#query = session.query(ipas.IceCrystal).all()[0:5]
#for mono in query:
#    print(mono.id, mono.phi, mono.r, mono.rand_orient, mono.aggs.ncrystals, mono.aggs.agg_phi)


AGGREGATE PROPERTIES ==============
(1,)
(2,)
(3,)


In [18]:
print('AGGREGATE PROPERTIES ==============')
query = session.query(ipas.IceCluster).filter(ipas.IceCluster.ncrystals == 3)

#query = session.query(clus.IceCluster)
for agg in query:
    print(agg.id, agg.ncrystals, agg.agg_phi, agg.agg_r, agg.id)

print('MONOMER PROPERTIES ================')
query = session.query(ipas.IceCrystal)
for mono in query.all():
    print(mono.id, mono.phi, mono.r, mono.rand_orient)

AGGREGATE PROPERTIES ==============
2 3 1.8740173820466801 59.37698841565121 2
6 3 2.3859356337384865 55.6787106938678 6
10 3 2.778599074182196 66.26468002911497 10
14 3 1.6747236078478691 62.99314501494472 14
18 3 1.6728977640739549 51.6818103407125 18
22 3 0.562322453228146 294.5122474931198 22
26 3 0.18439825476181357 211.27347388931824 26
30 3 0.36015275168692856 340.95968246474297 30
34 3 0.39139316984619915 244.6557059168472 34
38 3 0.2983764505124918 230.7597104241278 38
42 3 1.4532654870560684 60.4445862691194 42
46 3 1.7413433527872957 52.41042316482967 46
50 3 2.750634979565331 61.86103805993745 50
54 3 2.4786553214917566 70.60972644840311 54
58 3 2.705832162066122 65.44225636068938 58
62 3 0.13540536345156187 299.8724499109296 62
66 3 0.3313563839993581 318.3507886940626 66
70 3 0.4351117599762381 262.1617040476348 70
74 3 0.19861334526022234 250.1455844660929 74
78 3 0.16084564788548564 248.01778158792172 78
MONOMER PROPERTIES ================
1 0.01 9.999999999999996 True


/Users/vprzybylo/anaconda3/envs/IPAS/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as ipas.ice_crystal.IceCrystal, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)
[autoreload of ipas.ice_crystal failed: Traceback (most recent call last):
  File "/Users/vprzybylo/anaconda3/envs/IPAS/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/Users/vprzybylo/anaconda3/envs/IPAS/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/Users/vprzybylo/anaconda3/envs/IPAS/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/Users/vprzybylo/anaconda3/envs/IPAS/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
 

In [17]:
stmt = select('*').select_from(ipas.IceCluster)
result = session.execute(stmt).fetchall()

def result_dict(r):
    return dict(zip(r.keys(), r))

def result_dicts(rs): 
    return list(map(result_dict, rs))

result_dicts(result)

[{'id': 1,
  'ncrystals': 2,
  'points': b'\x80\x04\x95\x18\x03\x00\x00\x00\x00\x00\x00\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x02K\x0c\x86\x94h\x03\x8c\x05dtype\x94\x93\x94\x8c\x03V24\x94K\x00K\x01\x87\x94R\x94(K\x03\x8c\x01|\x94N\x8c\x01x\x94\x8c\x01y\x94\x8c\x01z\x94\x87\x94}\x94(h\x11h\x0c\x8c\x02f8\x94K\x00K\x01\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94bK\x00\x86\x94h\x12h\x18K\x08\x86\x94h\x13h\x18K\x10\x86\x94uK\x18K\x01K\x10t\x94b\x89B@\x02\x00\x00\xca`\xf0\x9c\xcfD7@\x88\xee{\xfd\xff~6@\xd9\x1d\xe3\xa2\x08\xa3@\xc0C\x0b\xe7\xed\xd0\xfb4@*\xf0\xf4\x8c[%D@\xecM*\xcd\x8d\x11#@6\x9c\x9c\xd2eS\xfe\xbfJ1\x8c73\x921@\x1a\x1f\x9aR\nvE@GI1\xf9>}6\xc0\x8fo?\xc7\x07\xf26\xc0]\xf9\xbb\x1bE\xc0@@\xc6\xf3\'J@44\xc0\xaf\xb0\xd6q\xdf^D\xc0\xd4\xdf\xc6\xe9\x9b\x9c"\xc0#\nG\x078f\x05@O\xb2O\x01;\x052\xc0\x92C\xc1\xd9\xcdXE\xc0GI1\xf9>}6@\x8fo?\xc7

In [27]:
file = open('instance_3radii_iceagg_allrand_50xtalstot_20phi_rall_300agg', 'rb')
b1 = pickle.load(file)

KeyboardInterrupt: 

In [ ]:
print(b1.rxs.shape)
print(np.max(b1.rxs))
print(np.max(b1.rys))
print(np.max(b1.rzs))

In [ ]:
#a=x b=y c=z

#dunnavan: a>= b >= c
#in our case z > y > x
phiba = b1.rxs/b1.rzs
phica = b1.rys/b1.rzs
#H, xedges, yedges = np.histogram2d(phica[10,:,2],phiba[10,:,2], normed=True)
#plt.imshow(H)
ax = sns.jointplot(x=phiba[1,:,2], y=phica[1,:,2], kind='hex')
ax = sns.jointplot(x=phiba[1,:,2], y=phica[1,:,2], kind='kde')
ax = sns.jointplot(x=phiba[19,:,2], y=phica[19,:,2], kind='hex', color='r')
ax = sns.jointplot(x=phiba[19,:,2], y=phica[19,:,2], kind='kde', color='r')



In [ ]:
n, bins, patches = plt.hist(phiba[10,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(phica[10,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)

In [ ]:
#axes rand plates
n, bins, patches = plt.hist(b1.rxs[1,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[1,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[1,:,2], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
#axes rand columns 
n, bins, patches = plt.hist(b1.rxs[19,:,2], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[19,:,2], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[19,:,2], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
n, bins, patches = plt.hist(b1.rxs[1,:,19], bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rys[1,:,19], bins='auto', color='r',
                            alpha=0.7, rwidth=0.85)
n, bins, patches = plt.hist(b1.rzs[1,:,19], bins='auto', color='g',
                            alpha=0.7, rwidth=0.85)

In [ ]:
phios = [0,5,10,15,19]
print(b1.phio)
phiba = b1.rxs/b1.rzs
phica = b1.rys/b1.rzs
for i in phios:
    print(b1.phio[i])
    if i > 9:
        color = 'r'
    else:
        color = 'b'
    sns.jointplot(x=phiba[i,:,20], y=phica[i,:,20], kind='kde', xlim=[0,1], ylim=[0,1], color=color)
    

In [ ]:
file = open('instance_3radii_iceagg_allflat_40xtalstot_20phi_r10', 'rb')
b1 = pickle.load(file)

In [ ]:
phiba = b1.rxs/b1.rzs
phica = b1.rys/b1.rzs
phios = [0,5,10,15,19]
print(b1.phio)
for i in phios:
    print(b1.phio[i])
    if i > 9:
        color = 'r'
    else:
        color = 'b'
    sns.jointplot(x=phiba[i,:,38], y=phica[i,:,38], kind='kde', xlim=[0,1], ylim=[0,1], color=color)
    
    

In [ ]:
#a_ellipse (rzs) to a_mono (length for columns, width for plates)

if phio <= 1.0 #plates
    ellipse_ratio = mean(rzs[0:20:5,:,:])/mean(width[0:20:5,:,:])
else: #columns
    ellipse_ratio = mean(rzs[0:20:5,:,:])/mean(length[0:20:5,:,:])

plt.plot(np.arange(2,39,1), ellipse_ratio)


In [ ]:
filename = 'instance_3radii_aggagg_allflat_40xtalstot_20phi_r10'
filehandler = open(filename, 'wb')
#%time pickle.dump(b1, filehandler)
#filehandler.close()
#print('finished!')

In [ ]:
b5.ch_req_shape = b5.ch_req.reshape(28, 20, 40)[:,:,:-1]
b5.ch_majorax_shape = b5.ch_majorax.reshape(28, 20, 40)[:,:,:-1]
b5.ch_minorax_shape = b5.ch_minorax.reshape(28, 20, 40)[:,:,:-1]
b5.chphi_shape = b5.chphi.reshape(28, 20, 40)[:,:,:-1]
b5.ch_dd_shape = b5.ch_dd.reshape(28, 20, 40)[:,:,:-1] 

In [ ]:
phioarr=np.logspace(-2, 2, num=20, dtype=None)#just columns (0,2); plates (-2,0)
#phioarr = phioarr[1:]
reqarr = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100,200,300,400,500,600,700,800,900,1000]
ncrystals=39

f1 = open('find_N_lookup_iceagg_flat_rall.dat',"w")
f2 = open('major_axis_lookup_iceagg_flat_rall.dat',"w")
f3 = open('minor_axis_lookup_iceagg_flat_rall.dat',"w")
f4 = open('dd_lookup_iceagg_flat_rall.dat',"w")

for i in range(len(reqarr)):
    for j in range(len(phioarr)):   
        for N in range((ncrystals)):
            #f1.write('%.3f\t %.1f\t %.4f\t %10.4f\t\n'%(b1.phioarr[j], reqarr[i], b1.chphi[N], b1.ch_req[N]))
            f1.write('{:8.4f} {:8.2f} {:8.4f} {:10.2f}\n'.format(b5.phioarr[j], reqarr[i], b5.chphi_shape[i,j,N], b5.ch_req_shape[i,j,N]))
            #f2.write('%.3f\t %.1f\t %.4f\t\n'%(b1.phioarr[j], reqarr[i], b1.ch_majorax[N]))
            f2.write('{:8.4f} {:8.2f} {:10.2f}\n'.format(b5.phioarr[j], reqarr[i], b5.ch_majorax_shape[i,j,N]))
            #print(b1.phioarr[j], reqarr[i], b1.ch_majorax[N])
           
            #minor_axis
            f3.write('{:8.4f} {:8.2f} {:8.4f}\n'.format(b5.phioarr[j], reqarr[i], b5.ch_minorax_shape[i,j,N]))
            #f3.write('%.3f\t %.1f\t %.4f\t\n'%(b1.phioarr[j], reqarr[i], b1.ch_minorax[N]))
            #density change
            f4.write("{:8.4f} {:8.2f} {:8.4f}\n".format(b5.phioarr[j], reqarr[i], b5.ch_dd_shape[i,j,N]))
            #f4.write('%5.3f\t %5.1f\t %.5f\t\n'%(b1.phioarr[j], reqarr[i], b1.ch_dd[N]))
            
f1.close()
f2.close()
f3.close()
f4.close()
